# Module 11: Advanced Topics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/11_Module.ipynb)

**Week 16: Image Pyramids, Panorama Stitching, Template Matching, Video Processing**

## Learning Objectives
- Understand image pyramid theory and applications
- Implement multi-resolution processing
- Create panoramic images using stitching
- Apply template matching for object detection
- Process real-time video streams

---
## ⚠️ IMPORTANT: Run All Cells in Order

**This notebook must be executed sequentially from top to bottom.**

- Click **Runtime → Run all** (or **Cell → Run All**)
- Do NOT skip cells or run them out of order
- Each cell depends on variables from previous cells

---

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
print(f"OpenCV version: {cv2.__version__}")

---
## 1. Image Pyramids

An **image pyramid** is a multi-scale representation of an image.

### Gaussian Pyramid

**Level 0**: Original image $G_0$

**Level i+1**: Downsample level i
$$
G_{i+1} = \text{downsample}(G_i * g)
$$

Where:
- $g$: Gaussian kernel (typically 5×5)
- downsample: Remove every other row and column

**Result**: Each level is 1/4 the size of previous

### Laplacian Pyramid

Stores **difference** between levels:
$$
L_i = G_i - \text{expand}(G_{i+1})
$$

Where expand: Upsample and blur

**Properties**:
- Reconstructable: $G_i = L_i + \text{expand}(G_{i+1})$
- Bandpass filtered
- Used for blending

### Applications

1. **Multi-scale processing**: Detect features at different scales
2. **Image blending**: Seamless panoramas
3. **Compression**: Store differences instead of full images
4. **Template matching**: Find objects at multiple scales

In [ ]:
# Image pyramids demonstrationprint("=" * 70)print("IMAGE PYRAMIDS")print("=" * 70)# Load imageimport urllib.requestfrom urllib.request import Request, urlopenurl = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/480px-Cat03.jpg'urllib.request.urlretrieve(url, 'test.jpg')img = cv2.imread('test.jpg')img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)# Create Gaussian pyramidnum_levels = 4gaussian_pyr = [img.copy()]print(f"\nBuilding Gaussian pyramid with {num_levels} levels...")for i in range(num_levels - 1):    img = cv2.pyrDown(img)    gaussian_pyr.append(img)    print(f"  Level {i+1}: {img.shape[1]}×{img.shape[0]}")# Create Laplacian pyramidlaplacian_pyr = []print(f"\nBuilding Laplacian pyramid...")for i in range(num_levels - 1):    # Expand next level    expanded = cv2.pyrUp(gaussian_pyr[i + 1], dstsize=(gaussian_pyr[i].shape[1], gaussian_pyr[i].shape[0]))    # Subtract to get Laplacian    laplacian = cv2.subtract(gaussian_pyr[i], expanded)    laplacian_pyr.append(laplacian)    print(f"  Level {i}: {laplacian.shape[1]}×{laplacian.shape[0]}")# Add the smallest levellaplacian_pyr.append(gaussian_pyr[-1])# Visualize Gaussian pyramidfig, axes = plt.subplots(1, num_levels, figsize=(16, 4))for i, gp in enumerate(gaussian_pyr):    axes[i].imshow(cv2.cvtColor(gp, cv2.COLOR_BGR2RGB))    axes[i].set_title(f'Gaussian Level {i}\n{gp.shape[1]}×{gp.shape[0]}')    axes[i].axis('off')plt.suptitle('Gaussian Pyramid', fontsize=14)plt.tight_layout()plt.show()# Visualize Laplacian pyramidfig, axes = plt.subplots(1, num_levels, figsize=(16, 4))for i, lp in enumerate(laplacian_pyr):    # Normalize for visualization    lp_vis = cv2.normalize(lp, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)    axes[i].imshow(cv2.cvtColor(lp_vis, cv2.COLOR_BGR2RGB))    axes[i].set_title(f'Laplacian Level {i}\n{lp.shape[1]}×{lp.shape[0]}')    axes[i].axis('off')plt.suptitle('Laplacian Pyramid', fontsize=14)plt.tight_layout()plt.show()# Reconstruct from Laplacian pyramidprint(f"\nReconstructing from Laplacian pyramid...")reconstructed = laplacian_pyr[-1]for i in range(num_levels - 2, -1, -1):    reconstructed = cv2.pyrUp(reconstructed, dstsize=(laplacian_pyr[i].shape[1], laplacian_pyr[i].shape[0]))    reconstructed = cv2.add(reconstructed, laplacian_pyr[i])# Compare original and reconstructedfig, axes = plt.subplots(1, 3, figsize=(15, 5))axes[0].imshow(cv2.cvtColor(gaussian_pyr[0], cv2.COLOR_BGR2RGB))axes[0].set_title('Original')axes[0].axis('off')axes[1].imshow(cv2.cvtColor(reconstructed, cv2.COLOR_BGR2RGB))axes[1].set_title('Reconstructed')axes[1].axis('off')# Differencediff = cv2.absdiff(gaussian_pyr[0], reconstructed)axes[2].imshow(cv2.cvtColor(diff, cv2.COLOR_BGR2RGB))axes[2].set_title('Difference (amplified)')axes[2].axis('off')plt.tight_layout()plt.show()print("\nKey Insight: Pyramids enable multi-scale image processing!")

## 2. Template Matching

**Template matching** finds occurrences of a template image within a larger image.

### Methods

**1. Square Difference**:
$$
R(x, y) = \sum_{x', y'} [T(x', y') - I(x + x', y + y')]^2
$$

**2. Normalized Square Difference**:
$$
R(x, y) = \frac{\sum_{x', y'} [T(x', y') - I(x + x', y + y')]^2}{\sqrt{\sum_{x', y'} T(x', y')^2 \cdot \sum_{x', y'} I(x + x', y + y')^2}}
$$

**3. Cross-Correlation**:
$$
R(x, y) = \sum_{x', y'} T(x', y') \cdot I(x + x', y + y')
$$

**4. Normalized Cross-Correlation** (best):
$$
R(x, y) = \frac{\sum_{x', y'} T(x', y') \cdot I(x + x', y + y')}{\sqrt{\sum_{x', y'} T(x', y')^2 \cdot \sum_{x', y'} I(x + x', y + y')^2}}
$$

### OpenCV Methods

- `TM_SQDIFF`: Square difference (minimum is best)
- `TM_SQDIFF_NORMED`: Normalized square difference
- `TM_CCORR`: Cross-correlation (maximum is best)
- `TM_CCORR_NORMED`: Normalized cross-correlation
- `TM_CCOEFF`: Correlation coefficient
- `TM_CCOEFF_NORMED`: Normalized correlation coefficient

**Recommendation**: Use `TM_CCOEFF_NORMED` (robust to brightness changes)

In [ ]:
# Template matching demonstration
print("=" * 70)
print("TEMPLATE MATCHING")
print("=" * 70)

# Load image and template (use a crop as template)
img_full = cv2.imread('test.jpg')
h, w = img_full.shape[:2]

# Create template (crop from image)
template = img_full[h//4:h//4+100, w//4:w//4+100]
th, tw = template.shape[:2]

print(f"\nImage size: {w}×{h}")
print(f"Template size: {tw}×{th}")

# All template matching methods
methods = [
    ('TM_CCOEFF', cv2.TM_CCOEFF),
    ('TM_CCOEFF_NORMED', cv2.TM_CCOEFF_NORMED),
    ('TM_CCORR', cv2.TM_CCORR),
    ('TM_CCORR_NORMED', cv2.TM_CCORR_NORMED),
    ('TM_SQDIFF', cv2.TM_SQDIFF),
    ('TM_SQDIFF_NORMED', cv2.TM_SQDIFF_NORMED)
]

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.ravel()

# Show original and template
axes[0].imshow(cv2.cvtColor(img_full, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(template, cv2.COLOR_BGR2RGB))
axes[1].set_title('Template')
axes[1].axis('off')

axes[2].axis('off')

print(f"\nTesting {len(methods)} matching methods:\n")

for idx, (name, method) in enumerate(methods):
    # Apply template matching
    result = cv2.matchTemplate(img_full, template, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    
    # For SQDIFF methods, minimum is best match
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
        match_val = min_val
    else:
        top_left = max_loc
        match_val = max_val
    
    bottom_right = (top_left[0] + tw, top_left[1] + th)
    
    print(f"{name:20s}: match={match_val:.4f}, location={top_left}")
    
    # Draw result
    img_display = img_full.copy()
    cv2.rectangle(img_display, top_left, bottom_right, (0, 255, 0), 3)
    
    axes[idx + 3].imshow(cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB))
    axes[idx + 3].set_title(f'{name}\nScore: {match_val:.3f}')
    axes[idx + 3].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Template matching finds objects at fixed scale!")
print("For scale-invariant detection, use feature matching instead.")

## 3. Image Stitching (Panorama)

**Image stitching** combines multiple overlapping images into a panorama.

### Algorithm Steps

1. **Detect features** in all images (SIFT, ORB)
2. **Match features** between image pairs
3. **Estimate homography** $H$ between images:
   $$
   \begin{bmatrix} x_2 \\ y_2 \\ 1 \end{bmatrix} = H \begin{bmatrix} x_1 \\ y_1 \\ 1 \end{bmatrix}
   $$
4. **RANSAC** to find robust homography
5. **Warp images** to common coordinate frame
6. **Blend** overlapping regions

### Homography Estimation

Need at least **4 point correspondences**.

Solve system:
$$
\begin{bmatrix}
x_1 & y_1 & 1 & 0 & 0 & 0 & -x_2 x_1 & -x_2 y_1 \\
0 & 0 & 0 & x_1 & y_1 & 1 & -y_2 x_1 & -y_2 y_1
\end{bmatrix}
\begin{bmatrix} h_1 \\ h_2 \\ h_3 \\ h_4 \\ h_5 \\ h_6 \\ h_7 \\ h_8 \end{bmatrix}
=
\begin{bmatrix} x_2 \\ y_2 \end{bmatrix}
$$

### RANSAC (Random Sample Consensus)

Robust estimation in presence of outliers:

1. **Sample** 4 random matches
2. **Compute** homography from sample
3. **Count** inliers (matches consistent with H)
4. **Repeat** N times
5. **Select** H with most inliers
6. **Recompute** H using all inliers

In [ ]:
# Panorama stitching demonstration
print("=" * 70)
print("PANORAMA STITCHING")
print("=" * 70)

# For demonstration, create two overlapping images from one
img_orig = cv2.imread('test.jpg')
h, w = img_orig.shape[:2]

# Create overlapping images (simulating panorama pieces)
img1 = img_orig[:, :int(w*0.6)]
img2 = img_orig[:, int(w*0.4):]

print(f"\nImage 1: {img1.shape[1]}×{img1.shape[0]}")
print(f"Image 2: {img2.shape[1]}×{img2.shape[0]}")
print(f"Overlap region: ~{int(w*0.2)} pixels")

# Detect ORB features
orb = cv2.ORB_create(nfeatures=2000)
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

print(f"\nFeatures detected:")
print(f"  Image 1: {len(kp1)} keypoints")
print(f"  Image 2: {len(kp2)} keypoints")

# Match features
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)

print(f"  Matches found: {len(matches)}")

# Extract matched points
if len(matches) > 4:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # Find homography using RANSAC
    H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    
    inliers = mask.ravel().tolist().count(1)
    print(f"\nRANSAC results:")
    print(f"  Inliers: {inliers} / {len(matches)}")
    print(f"  Homography matrix H:")
    print(H)
    
    # Warp img2 to img1's coordinate frame
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    
    # Calculate output size
    pts1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)
    pts2 = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
    pts2_transformed = cv2.perspectiveTransform(pts2, H)
    
    pts = np.concatenate((pts1, pts2_transformed), axis=0)
    [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
    
    # Translation to keep everything positive
    t = [-xmin, -ymin]
    Ht = np.array([[1, 0, t[0]], [0, 1, t[1]], [0, 0, 1]])
    
    # Warp and stitch
    result = cv2.warpPerspective(img2, Ht @ H, (xmax - xmin, ymax - ymin))
    result[t[1]:h1 + t[1], t[0]:w1 + t[0]] = img1
    
    # Visualize
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    axes[0, 0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('Image 1 (Left)')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title('Image 2 (Right)')
    axes[0, 1].axis('off')
    
    # Draw matches
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None,
                                   flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    axes[1, 0].imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    axes[1, 0].set_title(f'Feature Matches (top 50)\n{inliers} inliers total')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    axes[1, 1].set_title('Stitched Panorama')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nPanorama size: {result.shape[1]}×{result.shape[0]}")
else:
    print("\nNot enough matches for homography estimation")

print("\nKey Insight: Panoramas combine multiple views using feature matching!")

## 4. Video Processing

**Video** is a sequence of frames (images).

### Video Capture

```python
cap = cv2.VideoCapture(0)  # 0 = default camera
cap = cv2.VideoCapture('video.mp4')  # From file
```

### Reading Frames

```python
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Process frame
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
```

### Video Properties

- `CAP_PROP_FRAME_WIDTH`: Frame width
- `CAP_PROP_FRAME_HEIGHT`: Frame height
- `CAP_PROP_FPS`: Frames per second
- `CAP_PROP_FRAME_COUNT`: Total frames

### Video Writing

```python
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

for frame in frames:
    out.write(frame)

out.release()
```

### Real-time Processing Tips

1. **Reduce resolution**: Process smaller frames
2. **Skip frames**: Process every Nth frame
3. **Multi-threading**: Separate capture and processing
4. **GPU acceleration**: Use CUDA if available
5. **Optimize algorithms**: Use fast methods (ORB > SIFT)

In [ ]:
# Video processing example (conceptual code)
print("=" * 70)
print("VIDEO PROCESSING")
print("=" * 70)

print("""
Video processing typically requires:
1. Access to camera or video file
2. Real-time display window

In Colab/Jupyter, video capture is limited. 
Here's the standard workflow:

```python
import cv2

# Open camera
cap = cv2.VideoCapture(0)

# Check if opened successfully
if not cap.isOpened():
    print("Error: Cannot open camera")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"Video: {width}×{height} @ {fps} FPS")

while True:
    # Capture frame
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Can't receive frame")
        break
    
    # Process frame (example: convert to grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply edge detection
    edges = cv2.Canny(gray, 100, 200)
    
    # Display result
    cv2.imshow('Original', frame)
    cv2.imshow('Edges', edges)
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
```

Common video processing applications:
- Object detection and tracking
- Motion detection
- Face detection/recognition
- Optical flow
- Background subtraction
- Activity recognition
""")

print("\nKey Insight: Video = sequence of images processed in real-time!")

## Summary

### Key Concepts

1. **Image Pyramids**: Multi-scale representation
2. **Template Matching**: Find patterns at fixed scale
3. **Panorama Stitching**: Combine images using homography
4. **Video Processing**: Real-time frame-by-frame analysis

### Pyramid Applications

| Type | Use Case | Storage |
|------|----------|----------|
| Gaussian | Multi-scale detection | 1.33× original |
| Laplacian | Image blending | 1.33× original |
| Steerable | Orientation analysis | Varies |

### Mathematical Formulas

**Gaussian pyramid**:
$$
G_{i+1} = \text{downsample}(G_i * g)
$$

**Laplacian pyramid**:
$$
L_i = G_i - \text{expand}(G_{i+1})
$$

**Homography**:
$$
\begin{bmatrix} x' \\ y' \\ 1 \end{bmatrix} = H \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}
$$

**Template matching (normalized correlation)**:
$$
R(x,y) = \frac{\sum_{x',y'} T(x',y') \cdot I(x+x', y+y')}{\sqrt{\sum T^2 \cdot \sum I^2}}
$$

### OpenCV Functions

| Operation | Function | Key Parameters |
|-----------|----------|----------------|
| Pyramid down | `cv2.pyrDown(img)` | Reduces size by 2 |
| Pyramid up | `cv2.pyrUp(img)` | Increases size by 2 |
| Template match | `cv2.matchTemplate(img, tmpl, method)` | TM_CCOEFF_NORMED |
| Find homography | `cv2.findHomography(src, dst, method)` | RANSAC |
| Warp perspective | `cv2.warpPerspective(img, H, size)` | Uses 3×3 matrix |
| Video capture | `cv2.VideoCapture(source)` | 0 for camera |
| Video write | `cv2.VideoWriter(file, fourcc, fps, size)` | XVID, MP4V |

## Course Complete!

Congratulations! You've completed all 11 modules covering:

✅ Image fundamentals and color spaces  
✅ Filtering and enhancement  
✅ Geometric transformations  
✅ Edge detection  
✅ Feature detection and matching  
✅ Contours and shape analysis  
✅ Histograms and equalization  
✅ Image segmentation  
✅ Advanced topics (pyramids, panoramas, video)  

### Next Steps

1. **Practice**: Complete the capstone projects
2. **Explore**: Try OpenCV's advanced modules (DNN, tracking)
3. **Build**: Create your own computer vision applications
4. **Learn more**: Deep learning, 3D vision, SLAM

Happy coding! 🚀